In [1]:
!pip install langchain langchain-community langchain-core gradio

In [2]:
from langchain_community.llms import QianfanLLMEndpoint
import os

print("# 初始化千帆")
os.environ["QIANFAN_AK"]='######'
os.environ["QIANFAN_SK"]='######'

llm = QianfanLLMEndpoint(streaming=True,
                         model="ERNIE-3.5-8K",
                         temperature=0.1)

response = llm.invoke("上海春天一般哪个月开始？")
print(response)

# 初始化千帆


[WARNING] [05-05 17:06:07] base.py:624 [t:4567799296]: This key `messages` does not seem to be a parameter that the model `ERNIE-3.5-8K` will accept
[INFO] [05-05 17:06:07] oauth.py:222 [t:4567799296]: trying to refresh access_token for ak `CekH3f***`
[INFO] [05-05 17:06:08] oauth.py:237 [t:4567799296]: sucessfully refresh access_token


上海的春天一般从**3月开始**，但具体开始日期会因当年的气候情况而略有不同。根据气象学意义上的入春标准，立春之后连续5天日平均气温不低于10℃，即可判定为入春。

在春天，上海的气温逐渐升高，但仍然比较湿润，多阴雨天气。此时，上海的植物开始复苏，樱花、桃花、玉兰花等花卉盛开，城市变得更加美丽。此外，春季也是上海举办各种花卉展览和活动的季节，如上海国际花卉博览会等。

总之，上海的春天是一个充满生机和美丽的季节，值得大家去体验和欣赏。


In [3]:
from langchain import PromptTemplate, LLMChain
from langchain.chains import LLMRequestsChain
from langchain.schema import HumanMessage
from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
import gradio as gr
import json

# 此处不能设定streaming=True，否则无法激活函数回调
chat = QianfanChatEndpoint(model="ERNIE-3.5-8K",
                          temperature=0.1)

print("# 定义函数")
@tool
def search_ip(question:str, ip:str) -> str:
    """
    首先获取输入的IP地址的位置信息，然后回答输入的问题
    @param question: 问题
    @param ip: IP地址
    """
    prompt_template = """以下是IP地址'{ip}'的位置信息：
    >>> {requests_result} <<<
    根据以上位置信息，回答以下这个问题：
    >>> {question} <<<"""
    prompt = PromptTemplate(
        input_variables=["question", "ip", "requests_result"],
        template=prompt_template
    )
    chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=prompt))
    inputs = {
        "question": question,
        "ip": ip,
        "url": "https://api.songzixian.com/api/ip?dataSource=generic_ip&ip=" + ip
    }
    return chain.invoke(inputs)

@tool
def search_phone(question:str, phone:str) -> str:
    """
    首先获取输入的手机号码的归属地信息，然后回答输入的问题
    @param question: 问题
    @param phone: 手机号码
    """
    prompt_template = """以下是手机号码'{phone}'的归属地信息：
    >>> {requests_result} <<<
    根据以上归属地信息，回答以下这个问题：
    >>> {question} <<<"""
    prompt = PromptTemplate(
        input_variables=["question", "phone", "requests_result"],
        template=prompt_template
    )
    chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=prompt))
    inputs = {
        "question": question,
        "phone": phone,
        "url": "https://api.songzixian.com/api/phone-location?dataSource=phone_number_location&phoneNumber=" + phone
    }
    return chain.invoke(inputs)

functions=[convert_to_openai_tool(search_ip)['function'],convert_to_openai_tool(search_phone)['function']]
print(functions)

print("# 回调方法")
def get_response(message):
    print("# 绑定函数")
    result = chat.invoke([HumanMessage(content=message)],
                               functions=functions)
    print(result)

    function_call_info = result.additional_kwargs.get("function_call", None)
    print(function_call_info)
    
    if not function_call_info:
        print("# 直接返回")
        return result.content

    print("# 调用函数")
    function_name = function_call_info["name"]
    function_args = json.loads(function_call_info["arguments"])
    function_result = eval(function_name)(function_args)
    print(function_result)
    return function_result["output"]

def submit(message, chat_history):
    bot_message = get_response(message)
    # 保存历史对话记录，用于显示
    chat_history.append((message, bot_message))
    return "", chat_history

print("# 创建交互")
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) # 对话框
    msg = gr.Textbox(label="Prompt") # 输入框
    submitBtn = gr.Button("Submit") # 提交按钮
    clearBtn = gr.ClearButton([msg, chatbot]) # 清除按钮
    # 提交
    msg.submit(submit, inputs=[msg, chatbot], outputs=[msg, chatbot]) 
    submitBtn.click(submit, inputs=[msg, chatbot], outputs=[msg, chatbot])
    
gr.close_all()
demo.launch()

# 示例问题1：120.230.93.202属于哪个城市？
# 示例问题2：13816088620属于哪个运营商？

# 定义函数
[{'name': 'search_ip', 'description': 'search_ip(question: str, ip: str) -> str - 首先获取输入的IP地址的位置信息，然后回答输入的问题\n    @param question: 问题\n    @param ip: IP地址', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string'}, 'ip': {'type': 'string'}}, 'required': ['question', 'ip']}}, {'name': 'search_phone', 'description': 'search_phone(question: str, phone: str) -> str - 首先获取输入的手机号码的归属地信息，然后回答输入的问题\n    @param question: 问题\n    @param phone: 手机号码', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string'}, 'phone': {'type': 'string'}}, 'required': ['question', 'phone']}}]
# 回调方法
# 创建交互
Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


# 绑定函数
content='' additional_kwargs={'finish_reason': 'function_call', 'request_id': 'as-fcd0dz2mmg', 'object': 'chat.completion', 'search_info': [], 'function_call': {'name': 'search_ip', 'arguments': '{"question":"120.230.93.202属于哪个城市？","ip":"120.230.93.202"}'}} response_metadata={'token_usage': {'prompt_tokens': 194, 'completion_tokens': 63, 'total_tokens': 257}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'function_call', 'id': 'as-fcd0dz2mmg', 'object': 'chat.completion', 'created': 1714900176, 'result': '', 'is_truncated': False, 'need_clear_history': False, 'function_call': {'name': 'search_ip', 'thoughts': '用户想要知道一个特定IP地址的地理位置信息，我需要使用search_ip工具来获取这个信息。', 'arguments': '{"question":"120.230.93.202属于哪个城市？","ip":"120.230.93.202"}'}, 'usage': {'prompt_tokens': 194, 'completion_tokens': 63, 'total_tokens': 257}} id='run-0706ab6f-a21f-4352-bbb1-1f1a0f13c426-0'
{'name': 'search_ip', 'arguments': '{"question":"120.230.93.202属于哪个城市？","ip":"120.230.93.202"}'}
# 调用函数


[WARNING] [05-05 17:09:36] base.py:624 [t:123145751166976]: This key `messages` does not seem to be a parameter that the model `ERNIE-3.5-8K` will accept


{'question': '120.230.93.202属于哪个城市？', 'ip': '120.230.93.202', 'url': 'https://api.songzixian.com/api/ip?dataSource=generic_ip&ip=120.230.93.202', 'output': "根据提供的位置信息，IP地址'120.230.93.202'属于中国的广州市。"}
# 绑定函数


[WARNING] [05-05 17:09:44] base.py:624 [t:123145751166976]: This key `messages` does not seem to be a parameter that the model `ERNIE-3.5-8K` will accept


content='' additional_kwargs={'finish_reason': 'function_call', 'request_id': 'as-fatmgz1g3s', 'object': 'chat.completion', 'search_info': [], 'function_call': {'name': 'search_phone', 'arguments': '{"question":"13816088620属于哪个运营商？","phone":"13816088620"}'}} response_metadata={'token_usage': {'prompt_tokens': 191, 'completion_tokens': 55, 'total_tokens': 246}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'function_call', 'id': 'as-fatmgz1g3s', 'object': 'chat.completion', 'created': 1714900184, 'result': '', 'is_truncated': False, 'need_clear_history': False, 'function_call': {'name': 'search_phone', 'thoughts': "用户想要知道一个手机号码的归属地信息，我需要调用'search_phone'工具。", 'arguments': '{"question":"13816088620属于哪个运营商？","phone":"13816088620"}'}, 'usage': {'prompt_tokens': 191, 'completion_tokens': 55, 'total_tokens': 246}} id='run-5034ccc8-06fa-4743-9508-a3d712c97bc0-0'
{'name': 'search_phone', 'arguments': '{"question":"13816088620属于哪个运营商？","phone":"13816088620"}'}
# 调用函数
{'question': '13816088620属于